In [2]:
import pyshark
import glob, os
import numpy as np
import pandas as pd
import itertools
import time

In [3]:
# Global Variables
ROUTER = '78:d2:94:4d:ab:3e'
WIFI_DEVICES = ['ec:1a:59:e4:fd:41', 'ec:1a:59:e4:fa:09',
                'ec:1a:59:e5:02:0d', '14:91:82:24:dd:35',
                '60:38:e0:ee:7c:e5', '14:91:82:cd:df:3d',
                'b4:75:0e:0d:94:65', 'b4:75:0e:0d:33:d5',
                '94:10:3e:2b:7a:55', '30:8c:fb:3a:1a:ad',
                'd0:73:d5:26:b8:4c', 'd0:73:d5:26:c9:27',
                'ac:84:c6:97:7c:cc', 'b0:4e:26:c5:2a:41',
                '70:4f:57:f9:e1:b8', ROUTER]

DEVICE_TYPE = {'ec:1a:59:e4:fd:41' : 'camera',
               'ec:1a:59:e4:fa:09' : 'camera',
               'ec:1a:59:e5:02:0d' : 'camera',
               '14:91:82:24:dd:35' : 'plug',
               '60:38:e0:ee:7c:e5' : 'plug',
               '14:91:82:cd:df:3d' : 'plug',
               'b4:75:0e:0d:94:65' : 'plug',
               'b4:75:0e:0d:33:d5' : 'plug',
               '94:10:3e:2b:7a:55' : 'plug',
               '30:8c:fb:3a:1a:ad' : 'camera',
               'd0:73:d5:26:b8:4c' : 'bulb', 
               'd0:73:d5:26:c9:27' : 'bulb',
               'ac:84:c6:97:7c:cc' : 'camera', 
               'b0:4e:26:c5:2a:41' : 'bulb',
               '70:4f:57:f9:e1:b8' : 'plug',
                ROUTER : 'router'}

DEVICE_NAME = {'ec:1a:59:e4:fd:41' : 'Netcam1', 
               'ec:1a:59:e4:fa:09' : 'Netcam2',
               'ec:1a:59:e5:02:0d' : 'Netcam3',
               '14:91:82:24:dd:35' : 'Insight',
               '60:38:e0:ee:7c:e5' : 'Mini',
               '14:91:82:cd:df:3d' : 'Switch1',
               'b4:75:0e:0d:94:65' : 'Switch2',
               'b4:75:0e:0d:33:d5' : 'Switch3',
               '94:10:3e:2b:7a:55' : 'Switch4',
               '30:8c:fb:3a:1a:ad' : 'Dropcam',
               'd0:73:d5:26:b8:4c' : 'Lifx1', 
               'd0:73:d5:26:c9:27' : 'Lifx2',
               'ac:84:c6:97:7c:cc' : 'Kasa', 
               'b0:4e:26:c5:2a:41' : 'TpBulb',
               '70:4f:57:f9:e1:b8' : 'TpPlug',
                ROUTER : 'Router'}

TRAINING_TEST = {'ec:1a:59:e4:fd:41' : 'train', 
                 'ec:1a:59:e4:fa:09' : 'train',
                 'ec:1a:59:e5:02:0d' : 'test',
                 '14:91:82:24:dd:35' : 'train',
                 '60:38:e0:ee:7c:e5' : 'train',
                 '14:91:82:cd:df:3d' : 'train',
                 'b4:75:0e:0d:94:65' : 'train',
                 'b4:75:0e:0d:33:d5' : 'train',
                 '94:10:3e:2b:7a:55' : 'test',
                 '30:8c:fb:3a:1a:ad' : 'train',
                 'd0:73:d5:26:b8:4c' : 'train', 
                 'd0:73:d5:26:c9:27' : 'test',
                 'ac:84:c6:97:7c:cc' : 'test', 
                 'b0:4e:26:c5:2a:41' : 'train',
                 '70:4f:57:f9:e1:b8' : 'test',}

DATA_PKT_SUBTYPES = {32 : 'Data',
                     40 : 'QoS_Data',
                     44 : 'QoS_Null'}

FEATURES = ["Time", "PacketLength", "Duration", "SourceAddr", "DestAddr", "Subtype"]
SRC_DIR = './Source/'
DST_DIR = './Destination/'
PCAP_DIR = '/root/Documents/Thesis/PCAPS'
TIMING_PKT_NUMBER = 25000
DATA_FRAME_TYPE = '2'

path_name = os.getcwd()
DATE = path_name[path_name.rindex('/')+1:]
PROC_TIME = "wifi_processing_time_" + DATE + ".csv"

In [39]:
def get_mac_vendors():
    """
    Uses the macvendors.co API to lookup the vendors of Wi-Fi devices.
    Requires internet access.
    
    Parameters
    ----------
    None
    
    Output
    ------
    None
    
    Returns
    -------
    device_vendors (dict): keys(str) = WIFI_DEVICES MAC addresses, values(str) = vendor names
    """
    import json, requests

    MAC_LOOKUP_API = 'http://macvendors.co/api/'

    # Get JSON response from API
    vendors_json = []
    for addr in WIFI_DEVICES:
        response = requests.get(MAC_LOOKUP_API + addr).text
        vendors_json.append(response)

    # Extracting company from API response
    vendors = []
    for vendor_json in vendors_json:
        response = json.loads(vendor_json)
        company = str(response['result']['company']).split(' ',1)[0].capitalize()
        vendors.append(company)

    # Put device MAC addresses and vendors into dictionary
    device_vendors = dict(zip(WIFI_DEVICES, vendors))
    
    return device_vendors

path='/root/Documents/Thesis/Code/Source'
all_csvs = glob.glob(os.path.join(path, "*.csv"))

# Collect all csvs in one dataframe
df_from_each_file = (pd.read_csv(f) for f in all_csvs)
df = pd.concat(df_from_each_file, ignore_index=True, sort=False)

# Add device type, device ID of each packet
df["DeviceType"] = df["SourceAddr"].map(DEVICE_TYPE)
df["Name"] = df["SourceAddr"].map(DEVICE_NAME)

# Add whether device is a training or test device
df["Set"] = df["SourceAddr"].map(TRAINING_TEST)

# One-hot encode device type (response variable)
deviceType_series = pd.get_dummies(df["DeviceType"])
df = pd.concat([df, deviceType_series], axis=1)

# One-hot encode MAC vendors
df["Vendor"] = df["SourceAddr"].map(get_mac_vendors())
vendor_series = pd.get_dummies(df["Vendor"])
df = pd.concat([df, vendor_series], axis=1)

# One-hot encode packet subtype
df["Subtype"] = df["SubtypeNum"].map(DATA_PKT_SUBTYPES)
subtype_series = pd.get_dummies(df["Subtype"])
df = pd.concat([df, subtype_series], axis=1)    

# Count packets for each device
device_counts = df["Name"].value_counts()

In [91]:
df["Name"].value_counts()

Mini       104280
Router     103593
Dropcam     64568
Kasa        23753
Netcam3      4867
Netcam1      4446
Netcam2      4407
Switch2      3046
Switch1      2668
Switch3      2634
Insight      2556
Switch4      2206
Lifx2         627
TpPlug        587
Lifx1         540
TpBulb        202
Name: Name, dtype: int64

In [302]:
%%time
device_names = list(df["Name"].unique())
# device_names.remove("Router")

list_assoc_pkts = []

for device in device_names:
    print "Working on", device
    assoc_pkts = count_assoc_pkts(device)
    list_assoc_pkts.append(assoc_pkts)
    
combined_assoc_pkts = pd.concat(list_assoc_pkts)
df["Assoc_Packets"] = combined_assoc_pkts


Working on Switch2
Length = 3046
Working on Netcam2
Length = 4407
Working on Insight
Length = 2556
Working on Netcam1
Length = 4446
Working on TpPlug
Length = 587
Working on Switch4
Length = 2206
Working on Router
Length = 103593
Working on TpBulb
Length = 202
Working on Kasa
Length = 23753
Working on Mini
Length = 104280
Working on Switch1
Length = 2668
Working on Switch3
Length = 2634
Working on Lifx1
Length = 540
Working on Lifx2
Length = 627
Working on Dropcam
Length = 64568
Working on Netcam3
Length = 4867
CPU times: user 3min 31s, sys: 33 ms, total: 3min 31s
Wall time: 3min 31s


In [305]:
df

,Time,PacketLength,Duration,SourceAddr,DestAddr,SubtypeNum,DeviceType,Name,Set,bulb,...,Belkin,Dropcam,Lifi,Netgear,Tp-link,Subtype,Data,QoS_Data,QoS_Null,Assoc_Packets
0,1.538462e+09,78,202,b4:75:0e:0d:94:65,78:d2:94:4d:ab:3e,40,plug,Switch2,train,0,...,1,0,0,0,0,QoS_Data,0,1,0,1
1,1.538462e+09,78,202,b4:75:0e:0d:94:65,78:d2:94:4d:ab:3e,40,plug,Switch2,train,0,...,1,0,0,0,0,QoS_Data,0,1,0,1
2,1.538462e+09,78,202,b4:75:0e:0d:94:65,ec:1a:59:f1:fb:21,40,plug,Switch2,train,0,...,1,0,0,0,0,QoS_Data,0,1,0,1
3,1.538462e+09,78,202,b4:75:0e:0d:94:65,78:d2:94:4d:ab:3e,40,plug,Switch2,train,0,...,1,0,0,0,0,QoS_Data,0,1,0,1
4,1.538462e+09,78,202,b4:75:0e:0d:94:65,78:d2:94:4d:ab:3e,40,plug,Switch2,train,0,...,1,0,0,0,0,QoS_Data,0,1,0,1
5,1.538462e+09,78,202,b4:75:0e:0d:94:65,78:d2:94:4d:ab:3e,40,plug,Switch2,train,0,...,1,0,0,0,0,QoS_Data,0,1,0,1
6,1.538462e+09,80,0,b4:75:0e:0d:94:65,01:00:5e:7f:ff:fa,32,plug,Switch2,train,0,...,1,0,0,0,0,Data,1,0,0,1
7,1.538462e+09,78,202,b4:75:0e:0d:94:65,78:d2:94:4d:ab:3e,40,plug,Switch2,train,0,...,1,0,0,0,0,QoS_Data,0,1,0,1
8,1.538462e+09,76,0,b4:75:0e:0d:94:65,ff:ff:ff:ff:ff:ff,32,plug,Switch2,train,0,...,1,0,0,0,0,Data,1,0,0,1
9,1.538462e+09,78,202,b4:75:0e:0d:94:65,78:d2:94:4d:ab:3e,40,plug,Switch2,train,0,...,1,0,0,0,0,QoS_Data,0,1,0,1


In [298]:
def count_assoc_pkts(device):
    # Get the count of packets of a given device that are sent within a second of each other (associated packets)
    
    ASSOC_PKT_THRESHOLD = 1 # the threshold in seconds within which a packet will be considered an assoc_pkt

    # Extract time values of all packets belonging to a certain device
    df_device = df[df["Name"]==device]
    pkt_time_values = np.array(df_device["Time"].values)
    print "Length =", len(pkt_time_values)
    
    assoc_pkt_counts = []
    
    # Iterate through each packet of the device
    for pkt_index in range(0,len(df_device)):  

        # Create an array of size=len(pkt_time_values) that contains the time value of packet X
        pkt_time = np.full((len(pkt_time_values),),df_device.iloc[pkt_index]["Time"])

        # Calculate the time difference between packet X and all other packets
        diff = np.abs(np.subtract(pkt_time, pkt_time_values))

        # Calculate the count of packets that would be considered an assoc_pkt based on ASSOC_PKT_THRESHOLD
        assoc_pkts = (diff <= ASSOC_PKT_THRESHOLD).sum()
        assoc_pkt_counts.append(assoc_pkts)
        
    
    series = pd.Series(assoc_pkt_counts, index=df_device.index)
    return series

#         print "Count assoc_pkts = ", assoc_pkts

In [297]:
df["Assoc_Packets"] = assocpkts
df["Assoc_Packets"].iloc[sw1_indices]
# df["Assoc_Packets"].iloc[sw4_indices]

249076     1.0
249077    15.0
249078    15.0
249079    16.0
249080    16.0
249081    16.0
249082    16.0
249083    16.0
249084    16.0
249085    16.0
249086    16.0
249087    16.0
249088    16.0
249089    16.0
249090    16.0
249091    16.0
249092    14.0
249093     2.0
249094     2.0
249095     1.0
249096     1.0
249097     1.0
249098     1.0
249099     1.0
249100     1.0
249101     1.0
249102     1.0
249103     1.0
249104     2.0
249105     2.0
          ... 
251714     1.0
251715     1.0
251716     1.0
251717     1.0
251718     1.0
251719     1.0
251720     4.0
251721     4.0
251722     4.0
251723     4.0
251724     1.0
251725     1.0
251726     2.0
251727     2.0
251728     1.0
251729     1.0
251730     1.0
251731     1.0
251732     1.0
251733     1.0
251734     1.0
251735     2.0
251736     2.0
251737     1.0
251738     1.0
251739     1.0
251740     2.0
251741     2.0
251742     1.0
251743     1.0
Name: Assoc_Packets, Length: 2668, dtype: float64

In [35]:
# cap = pyshark.FileCapture('/root/Documents/Thesis/PCAPS/master.cap', only_summaries=False)


In [77]:
# Find frame type
subtypes = {"Mgmt": 0, "Control" : 0, "Data" : 0}
for pkt in cap:
    if pkt.wlan.fc_type == '0':
        subtypes["Mgmt"] += 1
    if pkt.wlan.fc_type == '1':
        subtypes["Control"] += 1
    if pkt.wlan.fc_type == '2':
        subtypes["Data"] += 1

# Print out value counts of subtypes
total = sum(subtypes.values())
for key, value in subtypes.iteritems():
    print key, ",", value, ",", value/float(total)
print total

qos_pkt = cap[169]
data_pkt = data_pkts[0]
QOS = '44'
pkt.wlan.fc_type_subtype == QOS

data_pkt.wlan.fc_type

In [68]:
test_pkt = data_pkts[0]
print test_pkt.wlan.field_names

for pkt in data_pkts:
#     print pkt.frame_info.get_field('encap_type')
    for i, field in enumerate(pkt.wlan.field_names):
        print pkt.wlan.field_names[i] + ": " + pkt.wlan.get_field(field)

: CCMP parameters
fc_frag: 0
sa_resolved: ac:84:c6:97:7c:cc
fc_type_subtype: 40
ra: 78:d2:94:4d:ab:3e
seq: 3224
addr: 78:d2:94:4d:ab:3e
ta: ac:84:c6:97:7c:cc
fc: 0x00008841
fc_protected: 1
da: 78:d2:94:4d:ab:3e
ccmp_extiv: 0x000000176C15
ta_resolved: ac:84:c6:97:7c:cc
qos_tid: 0
fc_moredata: 0
staa_resolved: ac:84:c6:97:7c:cc
fc_subtype: 8
flags: 0x00000041
da_resolved: 78:d2:94:4d:ab:3e
fc_version: 0
fc_order: 0
qos_priority: 0
qos_ack: 0x00000000
sa: ac:84:c6:97:7c:cc
duration: 202
fc_tods: 1
staa: ac:84:c6:97:7c:cc
wep_key: 0
qos: 0x00000000
addr_resolved: 78:d2:94:4d:ab:3e
fc_ds: 0x00000001
frag: 0
fc_fromds: 0
fc_pwrmgt: 0
bssid_resolved: 78:d2:94:4d:ab:3e
qos_txop_dur_req: 0
qos_amsdupresent: 0
fc_retry: 0
bssid: 78:d2:94:4d:ab:3e
ra_resolved: 78:d2:94:4d:ab:3e
qos_bit4: 0
fc_type: 2

: CCMP parameters
fc_frag: 0
sa_resolved: 14:91:82:cd:df:3d
fc_type_subtype: 40
ra: 78:d2:94:4d:ab:3e
seq: 4053
addr: 78:d2:94:4d:ab:3e
ta: 14:91:82:cd:df:3d
fc: 0x00008841
fc_protected: 1
da: 78:d2

In [28]:
def random_forest_classifier(X_train, y_train, X_test, y_test):
    time_start = time.time()
    
    randomforest = RandomForestClassifier(random_state=0, n_jobs=2)
    rf_model = randomforest.fit(X_train, y_train)

    preds = rf_model.predict(X_test)
    score = rf_model.score(X_test, y_test)
    
    time_elapsed = time.time() - time_start
    return {'Score' : score, 'Time' : time_elapsed}

In [36]:
from sklearn.ensemble import RandomForestClassifier

features_list = [
        # Packet info
        "PacketLength", "Duration", 
        
        # Vendor 
        "Belkin", "Dropcam", "Lifi", "Netgear", "Tp-link",
    
        # 802.11 Data subtype
        "Data", "QoS_Data", "QoS_Null"]

y_list = ["camera", "bulb", "plug"]

# Get combos for one vs one
combinations = [combination for combination in itertools.combinations(y_list, 2)]

for device_type in combinations:
    # Only use data with the two device types needed for one vs one classification
    # classes = combinations[0]
    pos_device_type = device_type[0]
    neg_device_type = device_type[1]

    df_1v1 = df[(df["DeviceType"]==pos_device_type) | (df["DeviceType"]==neg_device_type)]
#     print "Original df:", df.shape
#     print "1v1 df:", df_1v1.shape

    # Separate df into train and test sets
    df_train = df_1v1[df_1v1["Set"]=="train"]
    df_test = df_1v1[df_1v1["Set"]=="test"]

    X_train = df_train[features_list]
    X_test = df_test[features_list]
    y_train = df_train[pos_device_type]
    y_test = df_test[pos_device_type]

    rf_clf = random_forest_classifier(X_train, y_train, X_test, y_test)
    print "{} vs {}".format(pos_device_type, neg_device_type)
    print "Score: {} /t Time: {} sec".format(rf_clf['Score'], rf_clf['Time'])

camera vs bulb
Score: 0.187848326324 /t Time: 0.419248104095 sec
camera vs plug
Score: 0.36090153758 /t Time: 0.540331125259 sec
bulb vs plug
Score: 0.828362573099 /t Time: 0.511598110199 sec
